# Kaggle Deep Learning Overview

This notebook is for notes and exercise from the Kaggle Learning 2.0 Deep Learning Course.

1. [Introduction to Convolutions for Computer Vision Based DL](#conv)  
2. [Transfer Learning](#transfer)
3.  
4.  

<a id='conv'></a>
## Introduction to Convolutions for Computer Vision Based DL

Convolutions are a way for a computer to 'determine' information about an image. These convolutions are an array of values learned by deep learning algorithms that, when applied (mulitplied) to pixel values, can determine what that area/pixel is 'supposed' to be in a way. 

For example, given a 2x2 array, a [ [1.5,1.5], [-1.5,-1.5] ] convolution can determine if a square of 4 pixels (shaded from white-black) makes up a horizontal line.

Let's do another example. The below cell will print a raw image as well as the output from applying a basic convolution to the image.


In [ ]:
%reload_ext autoreload
%autoreload 2
from my_modules import dl_helpers as helpers
from termcolor import cprint # colored prints

horizontal_line_conv = [[ 1, 1],
                        [-1,-1]]
vert_line_conv = [[1, -1],
                  [1, -1]]

# load_my_image and visualize_conv are utility functions taken from the Kaggle/learntools repository
original_image = helpers.load_my_image('dog_1')
helpers.visualize_conv(original_image, horizontal_line_conv, 'Horizontal Line Conv')
helpers.visualize_conv(original_image, vert_line_conv, 'Vertical Line Conv')

## Convolutions and tensorflow
The below sections introduce and show us how to setup our environment for using convolutions, keras, and tensorflow.

#### How convolutions work with DL
The video below explains the basics of how convolutions work to 'learn' image sets and predict.  
https://youtu.be/ToBPiUlLFEY

#### How to setup code for convolution DL
The video below explains some of the basic setup for a pre-existing model and convolutions.  
https://youtu.be/sDG5tPtsbSA

### Image Setup


In [ ]:
# Choose images to work with for our example
from os.path import join

image_dir = './images/dogs/train/'
img_dict = {'dog_1' : "00a338a92e4e7bf543340dc849230e75.jpg",
            'dog_2' : "0b345d4f2434903c374ad8b8513a289b.jpg",
            'dog_3' : "0db44ddb42bf1f97de987abe2bf01839.jpg",
            'dog_4' : "01f8540fb1084107a6eb3e528f82c1aa.jpg"}

img_paths = [join(image_dir, file) for file in img_dict.values()]
img_paths

### Function to Read and Prep Images for Modeling

In [ ]:
import numpy as np
# conda install -c conda-forge tensorflow
from tensorflow.python.keras.applications.resnet import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return output

### Create Model with Pre-Trained Weights

In [ ]:
# conda install -c conda-forge keras
from tensorflow.keras.applications import ResNet50

my_model = ResNet50(weights='./pre-trained/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
test_data = read_and_prep_images(img_paths)
preds = my_model.predict(test_data)

### Visualize Predictions


In [ ]:
from my_modules.dl_helpers import decode_predictions
from IPython.display import Image, display

most_likely_labels = decode_predictions(preds, top=3)
for i, img_path in enumerate(img_paths):
    display(Image(img_path))
    cprint(f'Most Likely Label: {i + 1}', 'green')
    print(most_likely_labels[i])

<a id='transfer'></a>
## Transfer Learning
Transfer learning is a technique of taking an existing pre-trained model and cutting off the output layer so we can quickly and effiently apply the model to other, maybe more specific, problems. An example of this is taking our previously trained model and using it to classify whether an image looks 'urban' or 'rural'.

In our personal example, we'll be determining whether a photo is landscape or portrait. We'll be using a dataset of dog photos which are either horizontal or vertical. The below code will be very similar to the Kaggle example.

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = './pre-trained/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Setup a new Sequential model
my_new_model = Sequential()
# Add the original pre-trained data minus the top (output) layer)
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
# Add our own (Dense) output layer with our 2 classes and softmax activation (probabilities)
my_new_model.add(Dense(num_classes, activation='softmax'))

# Disable the first (pre-trained) layer from training
my_new_model.layers[0].trainable = False



In [5]:
# Compile the model
my_new_model.compile(optimize='sgd',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

### Compile Step Review
There were three arguments supplied to the compile step, let's review them:
- **Optimizer** determines how we determine the numerical values that make up the model. It can affect the resulting model and its predictions. 
- **Loss** determines what goal we optimize when determining numerical values in the model. It can affect the resulting model and predictions.
- **Metrics** determines only what we print out while the model is being build, but it doesn't affect the model itself.

### Fitting the Model
We have our training and validation sets in ```'./sideways_dogs/images/train``` and ```'./sideways_dogs/images/val```. These will be used when setting up ```train_generator``` and ```validation_generator```.

We have 220 training images and 217 validation images. We'll use a batch size of 10 and have 22 epochs.

In [7]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# our image size in px
image_size = 244

data_generator = ImageDataGenerator(preprocess_input)

train_path = './sideways_dogs/images/train'
val_path = './sideways_dogs/images/val'

train_generator = data_generator.flow_from_directory(
                                    directory=train_path,
                                    target_size=(image_size, image_size),
                                    batch_size=10,
                                    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
                                        directory=val_path,
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
/home/ttbot/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
22/22 [==============================] - 198s 9s/step - loss: 0.6078 - accuracy: 0.6955 - val_loss: 0.3421 - val_accuracy: 0.8750


## 3. Data Augmentation
**TODO:** original notes were lost, redo

In [ ]:
# data augmentation exercise

## 4. Model from Scratch
**TODO:** original notes were lost, redo

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

#
img_rows, img_cols = 28, 28
#
num_classes = 10

def prep_data(raw):
    """

    """
    #
    y = raw[:, 0]
    #
    out_y = keras.utils.to_categorical(y, num_classes)
    #
    x = raw[:,1:]
    num_images = raw.shape[0] #
    #
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    #
    out_x = out_x / 255
    return out_x, out_y

fashion_file = ""
#
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

# 
fashion_model = Sequential()
# 1st
fashion_model.add(Conv2D(12,
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(img_rows, img_cols, 1)))
# fashion_model.add(Conv2D(20,
                         kernel_size=(3,3),
                         activation='relu'))
fashion_model.add(Conv2D(20,
                         kernel_size=(3,3),
                         activation='relu'))
#
fashion_model.add(Flatten())
#
fashion_model.add(Dense(100, activation='relu'))
#
fashion_model.add(Dense(num_classes,
                        activation='softmax'))


In [ ]:
#
fashion_model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer='adam',
                      metrics=['accuracy'])

In [ ]:
# 
fashion_model.fit(x, y, batch_size = 100, epochs=4, validation_split=0.2)


**TODO** second model notes

In [ ]:
second_fashion_model = Sequential()
second_fashion_model.add(Conv2D(9,
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(img_rows, img_cols, 1)))
second_fashion_model.add(Conv2D(18,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Conv2D(36,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Conv2D(18,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Flatten())
second_fashion_model.add(Dense(90, activation='relu'))
second_fashion_model.add(Dense(num_classes, activation='softmax'))
second_fashion_model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer='adam',
                      metrics=['accuracy'])
second_fashion_model.fit(x, y, batch_size = 100, epochs=4, validation_split=0.2)

*TODO* final notes 